# Imports

In [8]:
import pandas as pd 
import torch
from torch.utils.data import Dataset, random_split
from typing import List, Dict, Union
from typing import Any, TypeVar
import pandas as pd
import os
import opendatasets as od

from transformers import AutoTokenizer, TrainingArguments 
from transformers import Trainer, AutoModelForCausalLM, IntervalStrategy

set a seed

In [9]:
torch.manual_seed(2137)

# Dataset Download

NOTE: you will need a kaggle API key for the following to work

In [4]:
import json

# Path to your JSON file
json_file_path = "kaggle.json"

# Open the file and read the content
try:
  with open(json_file_path, "r") as f:
    json_data = json.load(f)
except FileNotFoundError:
  print(f"Error: JSON file not found at {json_file_path}")
  exit(1)

# Access username and key from the JSON data
try:
  username = json_data["username"]
  key = json_data["key"]
except KeyError:
  print("Error: 'username' or 'key' key not found in JSON data")
  exit(1)

In [6]:
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = key

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/dsxavier/diagnoise-me'
# Using opendatasets let's download the data sets
od.download(dataset, "dataset")

100%|██████████| 191M/191M [00:41<00:00, 4.82MB/s] 


# Load Dataset

In [12]:
DATA_PATH = "dataset\\diagnoise-me\\diagnose_en_dataset.feather"
data = pd.read_feather(DATA_PATH)
data = data['Patient'].values

SEQ_LEN: int = 1024
SAMPLE_SIZE: int =  int(data.shape[0] * 0.01) #Just get only .01 fraction of els
_data = [el[:SEQ_LEN]  for el in data[:SAMPLE_SIZE]]

In [13]:
print(data)

['Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?'
 'Hi doctor, I am a 26 year old male. I am 5 feet and 9 inches tall and weigh 255 pounds. When I eat spicy\xa0food, I poop blood. Sometimes when I have constipation as well, I poop a little bit of blood. I am really scared that I have colon cancer. I do have diarrhea often. I\xa0do not have a\xa0family history of colon\xa0cancer. I got blood tests done last night. Please find my reports attached.'
 'Hello doctor, I am 48 years old. I am experiencing weak erection and difficulty in sustaining the same. This condition was observed 10 years back. Also, there is premature ejaculation. Other physical ailments that I have are, I am suffering from hypertension and taking Amlopres-L (Amlodipine and Lisinopril) for the last 10 years, high cholesterol and triglycerides. My cholesterol level is 225 and triglyceride is 2